In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as graph
import copy

In [2]:
data= pd.read_csv("/home/msc2/Documents/Academics 3rd Sem/MDSC-301/data/Mall_Customers_train.csv")
data.head(4)

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,138,Male,32,73,73
1,181,Female,37,97,32
2,131,Male,47,71,9
3,5,Female,31,17,40


In [3]:
data_model_in= data[['Gender', 'Age', 'Annual Income (k$)']]

def cat_to_num(s):
    if s== 'Male':
        return 1
    elif s== 'Female':
        return 0
    
data_model_in['Gender_num']= data_model_in['Gender'].apply(cat_to_num)
data_model_in

/home/msc2/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,Gender,Age,Annual Income (k$),Gender_num
0,Male,32,73,1
1,Female,37,97,0
2,Male,47,71,1
3,Female,31,17,0
4,Male,30,137,1
...,...,...,...,...
155,Male,19,81,1
156,Male,30,99,1
157,Male,67,62,1
158,Female,35,120,0


In [4]:
Age_mean= np.mean(data_model_in['Age'].values)
Age_std= np.std(data_model_in['Age'].values)

AnnualIncome_mean= np.mean(data_model_in['Annual Income (k$)'].values)
AnnualIncome_std= np.std(data_model_in['Annual Income (k$)'].values)

Gender_mean= np.mean(data_model_in['Gender_num'].values)
Gender_std= np.std(data_model_in['Gender_num'].values)

Gender_mean

0.4375

In [5]:
data_model_in['Age_norm']= (data_model_in['Age']- Age_mean)/Age_std
data_model_in['AnnualIncome_norm']= (data_model_in['Annual Income (k$)']- AnnualIncome_mean)/AnnualIncome_std
data_model_in['Gender_norm']= (data_model_in['Gender_num']- Gender_mean)/Gender_std

data_model_in

,Gender,Age,Annual Income (k$),Gender_num,Age_norm,AnnualIncome_norm,Gender_norm
0,Male,32,73,1,-0.509930,0.441971,1.133893
1,Female,37,97,0,-0.148918,1.351494,-0.881917
2,Male,47,71,1,0.573107,0.366178,1.133893
3,Female,31,17,0,-0.582132,-1.680248,-0.881917
4,Male,30,137,1,-0.654335,2.867365,1.133893
...,...,...,...,...,...,...,...
155,Male,19,81,1,-1.448561,0.745145,1.133893
156,Male,30,99,1,-0.654335,1.427287,1.133893
157,Male,67,62,1,2.017155,0.025107,1.133893
158,Female,35,120,0,-0.293322,2.223120,-0.881917


In [6]:
data_model_norm_in= data_model_in[['Gender_norm', 'Age_norm', 'AnnualIncome_norm']]

In [7]:
k_fold= 10
block_size= int(data_model_norm_in.shape[0]/ k_fold)
block_dict= {}
for i in range(k_fold):
    if i< k_fold- 1:
        block_dict[i]= data_model_norm_in.loc[i* block_size: (i+1)* block_size- 1]
    else:
        block_dict[i]= data_model_norm_in.loc[i* block_size: ]
        
block_dict[1]

,Gender_norm,Age_norm,AnnualIncome_norm
16,-0.881917,-0.293322,-1.642351
17,1.133893,0.067690,0.366178
18,1.133893,0.645309,-0.050687
19,-0.881917,0.717512,0.138797
20,-0.881917,-1.087549,-1.566558
21,-0.881917,0.934119,1.010423
22,-0.881917,-0.293322,0.479868
23,-0.881917,-0.293322,-1.604455
24,-0.881917,1.295131,0.517765
25,-0.881917,1.511738,-0.429655


In [8]:
CV_dict= {}
data_dict= {}
for i in range(k_fold):
    data_dict['validation']= block_dict[i]
    
    not_i= np.array(range(k_fold))
    not_i= np.delete(not_i, i)
    frames= []
    for j in not_i:
        frames.append(block_dict[j])
        
    data_dict['train']= pd.concat(frames)
    
    CV_dict[i]= copy.deepcopy(data_dict)
    
CV_dict[2]['validation']

,Gender_norm,Age_norm,AnnualIncome_norm
32,-0.881917,0.284297,-0.505448
33,-0.881917,-0.726537,-0.808622
34,-0.881917,0.717512,0.025107
35,-0.881917,0.717512,-1.073900
36,-0.881917,0.717512,-0.846519
37,1.133893,-0.509930,2.450500
38,-0.881917,1.872751,-0.884416
39,-0.881917,-0.726537,0.669352
40,-0.881917,0.789714,-0.694932
41,-0.881917,0.573107,2.223120


In [9]:
idx = CV_dict[1]['validation'].index.values

CV_dict[1]['train'].values- CV_dict[1]['validation'].loc[idx[0]].values

array([[ 2.01581052, -0.21660729,  2.08432257],
       [ 0.        ,  0.14440486,  2.99384514],
       [ 2.01581052,  0.86642917,  2.00852902],
       [ 0.        , -0.28880972, -0.03789677],
       [ 2.01581052, -0.36101215,  4.5097161 ],
       [ 2.01581052,  0.28880972,  2.27380644],
       [ 2.01581052, -0.57761945,  2.65277417],
       [ 0.        , -0.9386316 ,  1.47797418],
       [ 0.        , -1.15523889,  1.70535483],
       [ 0.        ,  1.66065591,  0.07579355],
       [ 0.        , -0.07220243,  1.51587096],
       [ 2.01581052,  1.58845348,  1.36428386],
       [ 0.        ,  1.37184618,  0.37896774],
       [ 0.        ,  0.21660729,  1.85694192],
       [ 0.        ,  0.21660729,  3.60019352],
       [ 0.        , -0.28880972,  0.94741935],
       [ 0.        ,  0.57761945,  1.13690322],
       [ 0.        , -0.43321458,  0.83372903],
       [ 0.        ,  1.01083403,  1.66745805],
       [ 0.        ,  1.01083403,  0.56845161],
       [ 0.        ,  1.01083403,  0.795

In [11]:
fold= 1;
np.sum((CV_dict[0]['train'].values- CV_dict[0]['validation'].loc[0].values)**2, axis= 1)

array([8.45481134e+00, 3.39388886e-01, 1.57728886e+00, 5.66201885e+00,
       8.43132510e+00, 6.47190570e+00, 4.11184695e+00, 8.29826930e+00,
       7.32748110e+00, 8.91036535e+00, 5.99715597e+00, 6.15132657e-01,
       3.61450898e-01, 8.57481244e+00, 1.17173910e+01, 5.43945205e+00,
       5.59189160e+00, 5.67439498e+00, 5.74388029e+00, 7.86796902e+00,
       7.23031155e+00, 4.03418882e+00, 1.14999598e+01, 4.16211274e+00,
       7.04511487e+00, 8.40894957e+00, 2.81488433e-01, 5.67569009e-02,
       8.82465445e-01, 5.35604099e+00, 4.12024896e+00, 4.85009570e+00,
       7.64031065e-01, 5.17719209e+00, 8.27220335e+00, 7.99380222e+00,
       4.15982861e+00, 5.45660840e+00, 1.68038822e+00, 1.93419538e-01,
       1.35755554e+00, 1.89588171e+00, 7.18892847e+00, 7.15326918e+00,
       6.95025928e+00, 4.63409095e+00, 9.08862609e+00, 4.22972598e+00,
       7.17755265e+00, 9.16528321e+00, 6.89991209e+00, 1.66233912e-01,
       3.64899489e-01, 1.11012760e+01, 1.77863270e+00, 6.06369135e+00,
      

In [12]:
df_temp = copy.deepcopy(CV_dict[0]['train'])
df_temp['distance'] = np.sum((CV_dict[0]['train'].values - CV_dict[0]['validation'].loc[0].values)**2, axis = 1)

df_temp.sort_values(by = 'distance')

,Gender_norm,Age_norm,AnnualIncome_norm,distance
127,1.133893,-0.509930,0.517765,0.005745
43,1.133893,-0.365525,0.631455,0.056757
91,1.133893,-0.365525,0.631455,0.056757
67,1.133893,-0.148918,0.631455,0.166234
159,1.133893,-0.870942,0.214590,0.182032
...,...,...,...,...
131,1.133893,1.800548,-1.604455,9.526166
69,-0.881917,2.089358,-0.088584,11.101276
83,-0.881917,2.017155,-0.543345,11.420499
38,-0.881917,1.872751,-0.884416,11.499960


In [13]:
idx = df_temp.sort_values(by= 'distance').index.values
idx

array([127,  43,  91,  67, 159,  55, 152, 133,  42,  17,  28,  68, 132,
       121,  27, 143,  80,  94,  84,  48, 107,  44, 142, 155, 156,  81,
        97,  56, 151,  18,  54,  70,  57, 138, 105, 102, 153, 116, 130,
       134, 101,  89, 141,  75, 110, 112,  37, 140, 114, 113, 124,  74,
        22,  46, 135, 139,  52, 111,  39,  78,  63,  96,  88,  85, 117,
       146,  86, 128, 108, 125, 144,  61,  90, 154,  47, 120, 145,  49,
       109,  45,  31,  53, 137, 122,  32,  19,  33, 106,  82, 150,  34,
        99, 147, 103,  95,  77,  26,  73, 104,  71, 129,  21, 157,  72,
       148,  66,  92,  60,  40,  59,  64,  58,  36, 100, 158,  98,  24,
       149,  76,  79, 126,  87,  35,  51,  50,  23, 136, 118,  41,  20,
        16,  29, 123,  25, 119,  93,  62, 115,  65, 131,  69,  83,  38,
        30])

In [14]:
K = 5

K_idx = idx[0: K]

mean = 0
sum= 0
for i in K_idx:
    sum += data.loc[i]['Spending Score (1-100)']

mean= sum/ K

mean

48.2

In [15]:
nearest = pd.DataFrame(columns = data.columns)
for i in K_idx:
    nearest.loc[i] = data.loc[i]
    
nearest

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
127,142,Male,32,75,93
43,150,Male,34,78,90
91,159,Male,34,78,1
67,157,Male,37,78,1
159,121,Male,27,67,56


In [16]:
model_prediction = mean

actual_prediction = data.loc[0]['Spending Score (1-100)']

pred_error = actual_prediction - model_prediction

validation_cost = pred_error**2
pred_error

24.799999999999997